# Pandas: limpeza e tratamento de dados

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Conhecendo os dados

In [ ]:
import pandas as pd
import json
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
url = 'https://github.com/lukaswyllis/formacao_python_para_data_science/raw/refs/heads/master/dados-curso8/dataset-telecon.json'

In [ ]:
 dados_churn = pd.read_json(url)

In [ ]:
dados_churn

In [ ]:
dados_churn['conta'][0]

In [ ]:
pd.json_normalize(dados_churn['conta'])

In [ ]:
pd.json_normalize(dados_churn['telefone'])

In [ ]:
with open('/content/drive/MyDrive/Dados cursos/Formação Python para Data Science/dados-curso8/dataset-telecon.json') as f:
  json_bruto = json.load(f)

json_bruto

In [ ]:
dados_normalizados = pd.json_normalize(json_bruto)

In [ ]:
dados_normalizados

In [ ]:
data = {
    "empresa": "alura",
    "funcionarios": [
        {"nome": "Alice", "endereço": {"cidade": "São Paulo", "estado": "SP"}},
        {"nome": "Bob", "endereço": {"cidade": "Rio de Janeiro", "estado": "RJ"}}
    ]
}

df = pd.json_normalize(data, record_path='funcionarios', meta = "empresa", errors='ignore')
df

In [ ]:
json_desafio = {
  "nome": "João",
  "idade": 28,
  "enderecos": [
    {
      "tipo": "casa",
      "rua": "Rua A",
      "numero": 123,
      "cidade": "São Paulo"
    },
    {
      "tipo": "trabalho",
      "rua": "Rua B",
      "numero": 456,
      "cidade": "Rio de Janeiro"
    }
  ]
}

df_desafio = pd.json_normalize(json_desafio, record_path='enderecos', meta=['nome', 'idade'])
df_desafio


## Transformação inicial dos dados

In [ ]:
dados_normalizados.info()

In [ ]:
# retorna erro = dados_normalizados['conta.cobranca.Total'] = dados_normalizados['conta.cobranca.Total'].astype(float)

In [ ]:
dados_normalizados[dados_normalizados['conta.cobranca.Total'] == ' '][
    ['cliente.tempo_servico', 'conta.contrato', 'conta.cobranca.mensal', 'conta.cobranca.Total']
    ]

In [ ]:
idx = dados_normalizados[dados_normalizados['conta.cobranca.Total'] == ' '][
    ['cliente.tempo_servico', 'conta.contrato', 'conta.cobranca.mensal', 'conta.cobranca.Total']
    ].index

In [ ]:
dados_normalizados.loc[idx, 'conta.cobranca.Total'] = dados_normalizados.loc[idx, 'conta.cobranca.mensal'] * 24

In [ ]:
dados_normalizados.loc[idx, 'cliente.tempo_servico'] = 24

In [ ]:
dados_normalizados.loc[idx, ['cliente.tempo_servico', 'conta.contrato', 'conta.cobranca.mensal', 'conta.cobranca.Total']]

In [ ]:
dados_normalizados['conta.cobranca.Total'] = dados_normalizados['conta.cobranca.Total'].astype(float)

In [ ]:
dados_normalizados.info()

In [ ]:
for col in dados_normalizados.columns:
  print(f'Coluna: {col}')
  print(dados_normalizados[col].unique())
  print('-' * 100)

In [ ]:
dados_normalizados.query('Churn == ""')

In [ ]:
dados_sem_vazio = dados_normalizados[dados_normalizados['Churn'] != ''].copy()

In [ ]:
dados_sem_vazio.info()

In [ ]:
dados_sem_vazio.reset_index(drop=True, inplace=True)

In [ ]:
dados_sem_vazio.info()

In [ ]:
json_desafio2 = {
  "pessoas": [
    {
      "nome": "João",
      "idade": "25",
      "endereco": {
        "rua": "Rua A",
        "numero": 123,
        "cidade": "São Paulo"
      },
      "telefones": [
        "11 1111-1111",
        "11 2222-2222"
      ]
    },
    {
      "nome": "Maria",
      "idade": 30,
      "endereco": {
        "rua": "",
        "numero": 456,
        "cidade": "Rio de Janeiro"
      },
      "telefones": [
        "21 3333-3333"
      ]
    }
  ]
}

df_desafio2 = pd.json_normalize(json_desafio2, record_path='pessoas')
df_desafio2['idade'] = df_desafio2['idade'].astype(int)
df_desafio2 = df_desafio2[df_desafio2['endereco.rua'] != '']
df_desafio2

## Utilizando dados duplicados e nulos

In [ ]:
dados_sem_vazio.duplicated()

In [ ]:
dados_sem_vazio.duplicated().sum()

In [ ]:
filtro_duplicadas = dados_sem_vazio.duplicated()
filtro_duplicadas.sum()

In [ ]:
dados_sem_vazio[filtro_duplicadas]

In [ ]:
dados_sem_vazio.drop_duplicates(inplace=True)

In [ ]:
dados_sem_vazio.duplicated().sum()

In [ ]:
dados_sem_vazio.isna()

In [ ]:
dados_sem_vazio.isna().sum()

In [ ]:
dados_sem_vazio.isna().sum().sum()

In [ ]:
dados_sem_vazio[dados_sem_vazio.isna().any(axis=1)]

In [ ]:
filtro = dados_sem_vazio['cliente.tempo_servico'].isna()

In [ ]:
dados_sem_vazio[filtro][['cliente.tempo_servico', 'conta.cobranca.mensal', 'conta.cobranca.Total']]

In [ ]:
np.ceil(5957.90/90.45)

In [ ]:
dados_sem_vazio['cliente.tempo_servico'].fillna(
    np.ceil(
        dados_sem_vazio['conta.cobranca.Total'] / dados_sem_vazio['conta.cobranca.mensal']
    ), inplace=True
)

In [ ]:
dados_sem_vazio[filtro][['cliente.tempo_servico', 'conta.cobranca.mensal', 'conta.cobranca.Total']]

In [ ]:
dados_sem_vazio.isna().sum()

In [ ]:
dados_sem_vazio['conta.contrato'].value_counts()

In [ ]:
colunas_dropar = ['conta.contrato', 'conta.faturamente_eletronico', 'conta.metodo_pagamento']

In [ ]:
dados_sem_vazio[colunas_dropar].isna()

In [ ]:
dados_sem_vazio[colunas_dropar].isna().any(axis=1).sum()

In [ ]:
df_sem_nulos = dados_sem_vazio.dropna(subset=colunas_dropar).copy().reset_index()

In [ ]:
df_sem_nulos

In [ ]:
df_sem_nulos.isna().sum()

## Desafio: tratando uma base de dados

In [ ]:
with open('/content/drive/MyDrive/Dados cursos/Formação Python para Data Science/dados-curso8/cursos_cadastrados.json') as f:
  json_desafio_aula3 = json.load(f)

In [ ]:
df_desafio3 = pd.json_normalize(json_desafio_aula3)
df_desafio3

In [ ]:
df_desafio3[df_desafio3.isna().any(axis=1)]

In [ ]:
df_desafio3.dropna(inplace=True)

In [ ]:
df_desafio3

In [ ]:
df_desafio3.duplicated()

In [ ]:
df_desafio3.drop_duplicates(inplace=True)

In [ ]:
df_desafio3

In [ ]:
# Substitui strings vazias por valores nulos em todo o dataframe
df_desafio3.replace('', pd.NA, inplace=True)

In [ ]:
df_desafio3

In [ ]:
df_desafio3.dropna(inplace=True)

In [ ]:
df_desafio3

In [ ]:
df_desafio3.info()

In [ ]:
# Converte a concluintes para o tipo inteiro
df_desafio3['concluintes'] = df_desafio3['concluintes'].astype(int)

# Converte a coluna data_inicio e data_conclusao para o tipo datetime
df_desafio3['data_inicio'] = pd.to_datetime(df_desafio3['data_inicio'])
df_desafio3['data_conclusao'] = pd.to_datetime(df_desafio3['data_conclusao'])

# Convertendo a coluna preço para o tipo float
df_desafio3['preco'] = df_desafio3['preco'].astype(float)

In [ ]:
df_desafio3.info()

## Lidando com outliers

In [ ]:
df_sem_nulos.describe()

In [ ]:
sns.boxplot(x=df_sem_nulos['cliente.tempo_servico'])

In [ ]:
q1 = df_sem_nulos['cliente.tempo_servico'].quantile(.25)
q3 = df_sem_nulos['cliente.tempo_servico'].quantile(.75)
iqr = q3-q1
lim_inferior = q1-1.5*iqr
lim_superior = q3+1.5*iqr

In [ ]:
outliers_index = (df_sem_nulos['cliente.tempo_servico'] < lim_inferior) | (df_sem_nulos['cliente.tempo_servico'] > lim_superior)

In [ ]:
outliers_index

In [ ]:
df_sem_nulos[outliers_index]['cliente.tempo_servico']

In [ ]:
df_sem_out = df_sem_nulos.copy().reset_index()

In [ ]:
df_sem_out

In [ ]:
df_sem_out[outliers_index]['cliente.tempo_servico']

In [ ]:
df_sem_out.loc[outliers_index, 'cliente.tempo_servico'] = np.ceil(
    df_sem_out.loc[outliers_index, 'conta.cobranca.Total'] /
    df_sem_out.loc[outliers_index, 'conta.cobranca.mensal']
)

In [ ]:
sns.boxplot(x=df_sem_out['cliente.tempo_servico'])

In [ ]:
df_sem_out[outliers_index][['cliente.tempo_servico', 'conta.cobranca.mensal', 'conta.cobranca.Total']]

In [ ]:
# Z-score

# Dados de exemplo
data = np.array([10, 20, 30, 40, 150, 50, 60, 70, 80, 90, 100, 350])

# Cálculo do z-score
z_scores = (data - np.mean(data)) / np.std(data)

# Limite para considerar um dado como outlier
limite = 3

# Identificação dos outliers
outliers = data[np.abs(z_scores) > limite]

print("Outliers encontrados:", outliers)

In [ ]:
# Regra dos três sigmas

# Criar um array com os dados
dados = np.array([0, 10, 12, 13, 15, 16, 18, 20, 22, 25, 30, 35, 40, 50, 350])

# Calcular a média e o desvio padrão do conjunto de dados
media = np.mean(dados)
desvio_padrao = np.std(dados)

# Definir o limite superior e inferior para identificar os outliers
limite_superior = media + (3 * desvio_padrao)
limite_inferior = media - (3 * desvio_padrao)

# Identificar os outliers no conjunto de dados
outliers = dados[(dados > limite_superior) | (dados < limite_inferior)]

print("Outliers:", outliers)

In [ ]:
# Análise de dispersão

# Dados de exemplo
data = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90, 100])

# Criação do boxplot
fig, ax = plt.subplots()
ax.boxplot(data)

# Identificação dos outliers
outliers = data[(data < np.percentile(data, 25) - (1.5 * (np.percentile(data, 75) - np.percentile(data, 25)))) |
                (data > np.percentile(data, 75) + (1.5 * (np.percentile(data, 75) - np.percentile(data, 25))))]

print("Outliers encontrados:", outliers)

In [ ]:
df_sem_out[outliers_index]['cliente.tempo_servico']

In [ ]:
q1 = df_sem_out['cliente.tempo_servico'].quantile(.25)
q3 = df_sem_out['cliente.tempo_servico'].quantile(.75)
iqr = q3-q1
lim_inferior = q1-1.5*iqr
lim_superior = q3+1.5*iqr
outliers_index = (df_sem_out['cliente.tempo_servico'] < lim_inferior) | (df_sem_out['cliente.tempo_servico'] > lim_superior)

In [ ]:
outliers_index

In [ ]:
df_sem_out[outliers_index]['cliente.tempo_servico']

In [ ]:
df_sem_out = df_sem_out[~outliers_index]
df_sem_out

In [ ]:
sns.boxplot(x=df_sem_out['cliente.tempo_servico'])

In [ ]:
df_sem_out.reset_index(drop=True, inplace=True)

In [ ]:
# df_sem_out.drop('level_0')
df_sem_out.drop(columns=['level_0', 'index'], inplace=True)

In [ ]:
df_sem_out

## Desafio: identificando fraudes

In [ ]:
df = pd.DataFrame({
    'ID da transação': range(1, 31),
    'Valor da transação': [100, 200, 150, 500, 300, 913, 250, 400, 200, 150,
                           200, 200, 400, 300, 150, 301, 805, 300, 400, 250,
                           150, 100, 500, 600, 200, 350, 100, 250, 800, 250],
    'Data da transação': pd.date_range(start='2022-01-01', end='2022-01-30', freq='D'),
    'Local da transação': ['São Paulo, Brasil', 'Rio de Janeiro, Brasil', 'Belo Horizonte, Brasil', 'São Paulo, Brasil',
                           'São Paulo, Brasil', 'Nova Iorque, EUA', 'São Paulo, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil',
                           'Rio de Janeiro, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil',
                           'São Paulo, Brasil', 'São Paulo, Brasil', 'Los Angeles, EUA', 'São Paulo, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil',
                           'São Paulo, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil',
                           'São Paulo, Brasil', 'São Paulo, Brasil', 'Miami, EUA', 'São Paulo, Brasil']
})
df

In [ ]:
def identifica_outliers(coluna: str, df: pd.DataFrame):
  q1 = df[coluna].quantile(.25)
  q3 = df[coluna].quantile(.75)
  iqr = q3-q1
  lim_inferior = q1-1.5*iqr
  lim_superior = q3+1.5*iqr
  outliers_index = (df[coluna] < lim_inferior) | (df[coluna] > lim_superior)
  return outliers_index

In [ ]:
identifica_outliers('Valor da transação', df)

## Trabalhando com variáveis categóricas

In [ ]:
df_sem_out.drop('id_cliente', axis=1)

In [ ]:
df_sem_id = df_sem_out.drop('id_cliente', axis=1).copy()

In [ ]:
df_sem_id

In [ ]:
mapeamento = {
    'nao': 0,
    'sim': 1,
    'masculino': 0,
    'feminino': 1,
}

In [ ]:
for col in df_sem_id.columns:
  print(f'Coluna: {col}')
  print(df_sem_id[col].unique())
  print('-' * 100)

In [ ]:
colunas = ['telefone.servico_telefone', 'Churn', 'cliente.parceiro', 'cliente.dependentes', 'conta.faturamente_eletronico', 'cliente.genero']

In [ ]:
df_sem_id[colunas] = df_sem_id[colunas].replace(mapeamento)
df_sem_id

In [ ]:
for col in df_sem_id.columns:
  print(f'Coluna: {col}')
  print(df_sem_id[col].unique())
  print('-' * 100)

In [ ]:
s = pd.Series(list('abca'))
s

In [ ]:
pd.get_dummies(s)

In [ ]:
df_sem_id.info()

In [ ]:
pd.get_dummies(df_sem_id)

In [ ]:
df_dummies = pd.get_dummies(df_sem_id).copy()

In [ ]:
df_dummies

In [ ]:
df_dummies.columns

In [ ]:
df_dummies.info()